In [17]:
import numpy as np
from numba import njit
import time


In [18]:
@njit(fastmath=True)
def calcPay(Str, qvec, rvec, c):
    """Calculate payoffs and cooperation rate for given strategies"""
    n = Str.shape[0]  # Number of players
    m = len(rvec)     # Number of states
    
    # Total number of states: m * 2^n
    total_states = m * (1 << n)
    
    # PART I: Create all possible states
    PossState = np.zeros((total_states, n + 1), dtype=np.int32)
    
    j = 0
    for env_state in range(m):  # Environment states 1..m
        for i in range(1 << n):  # 2^n possible action combinations
            PossState[j, 0] = env_state
            
            # Convert i to binary representation for actions
            temp = i
            for k in range(n-1, -1, -1):
                PossState[j, k + 1] = temp % 2
                temp //= 2
            
            j += 1
    
    # Calculate payoffs for each state
    piRound = np.zeros((total_states, n))
    
    for i in range(total_states):
        State = PossState[i, :]
        
        # Count cooperators
        nrCoop = 0
        for k in range(1, n + 1):
            nrCoop += State[k]
        
        # Get multiplication factor for this state
        r_val = rvec[State[0]]
        
        for j in range(n):
            piRound[i, j] = nrCoop * r_val / n - State[j + 1] * c
    
    # PART II: Create transition matrix
    M = np.zeros((total_states, total_states))
    ep = 0.001
    
    # Add errors to strategies
    Str_err = Str * (1 - ep) + (1 - Str) * ep
    
    for row in range(total_states):
        StOld = PossState[row, :]
        
        # Count cooperators in old state
        nrCoop = 0
        for k in range(1, n + 1):
            nrCoop += StOld[k]
        
        # Get next environment state
        EnvNext = qvec[nrCoop]
        
        for col in range(total_states):
            StNew = PossState[col, :]
            
            if StNew[0] == EnvNext:
                trpr = 1.0
                
                for i_player in range(n):
                    iCoopOld = StOld[i_player + 1]
                    
                    # Calculate strategy index
                    others_coop = nrCoop - iCoopOld

                    if iCoopOld == 1:
                        index = (n - 1) - others_coop
                    else:
                        index = (2 * n - 1) - others_coop
                    
                    pval = Str_err[i_player, index]

                    iCoopNext = StNew[i_player + 1]
                    
                    # Probability player chooses iCoopNext
                    if iCoopNext == 1:
                        trpr *= pval
                    else:
                        trpr *= (1.0 - pval)
            else:
                trpr = 0.0
            
            M[row, col] = trpr
    
    # Approximate stationary distribution by power iteration
    v = np.ones(total_states) / total_states
    
    for iteration in range(10000):
        v_new = np.zeros(total_states)
        
        # Matrix multiplication: v_new = v @ M
        for i in range(total_states):
            for j in range(total_states):
                v_new[i] += v[j] * M[j, i]
        
        # Check convergence
        converged = True
        for i in range(total_states):
            if abs(v_new[i] - v[i]) > 1e-12:
                converged = False
                break
        
        if converged:
            break
        
        v = v_new
    
    # Normalize
    v_sum = 0.0
    for i in range(total_states):
        v_sum += v[i]
    v = v / v_sum
    
    # Calculate expected payoffs
    pivec = np.zeros(n)
    for j in range(n):
        for i in range(total_states):
            pivec[j] += v[i] * piRound[i, j]
    
    # Calculate cooperation rate
    cvec = 0.0
    for i in range(total_states):
        state_coop = 0.0
        for j in range(1, n + 1):
            state_coop += PossState[i, j]
        cvec += v[i] * state_coop
    
    cvec = cvec / n
    
    return pivec, cvec

In [19]:
# ==================== HELPER FUNCTIONS ====================

@njit(fastmath=True)
def comb(n, k):
    if k < 0 or k > n:
        return 0.0
    if k == 0 or k == n:
        return 1.0
    
    # Use logarithms to avoid overflow
    result = 0.0
    for i in range(1, k + 1):
        result += np.log(n - k + i) - np.log(i)
    return np.exp(result)

@njit(fastmath=True)
def CalcBinom(N, n):
    """Binomial coefficient calculations"""
    bm = np.zeros((N, n + 1))
    
    for row in range(N):
        i = row + 1  # number of strategy 1 players
        
        for col in range(n):
            j = col + 1  # number of strategy 1 co-players
            
            if j > i or (n - j) > (N - i):
                bm[row, col] = 0.0
            else:
                numerator = comb(i - 1, j - 1) * comb(N - i, n - j)
                denominator = comb(N - 1, n - 1)
                bm[row, col] = numerator / denominator
    
    return bm

@njit(fastmath=True)
def get_strategies(n):
    """Fast strategy generation using bit operations"""
    ns = 1 << (2 * n)  # 2^(2*n)
    Str = np.zeros((ns, 2 * n), dtype=np.float64)
    
    for i in range(ns):
        temp = i
        for j in range(2*n-1, -1, -1):
            Str[i, j] = temp % 2
            temp //= 2
    
    return Str


In [20]:
@njit(fastmath=True)
def CalcRho(S1, S2, PayH, N, n, qvec, rvec, c, beta, binom, Str):
    """
    Fixation probability calculation
    """
    alpha = np.zeros(N - 1)
    
    # Calculate payoffs for different group compositions
    Pay = np.zeros((n + 1, 2))
    Pay[n, 0] = PayH[S1]  # All S1
    Pay[0, 1] = PayH[S2]  # All S2
    
    st1 = Str[S1]
    st2 = Str[S2]
    
    # Mixed groups
    for nMut in range(1, n):
        # Create mixed group
        group_strats = np.zeros((n, 2 * n))
        for k in range(nMut):
            group_strats[k] = st1
        for k in range(nMut, n):
            group_strats[k] = st2
        
        pivec, _ = calcPay(group_strats, qvec, rvec, c)
        Pay[nMut, 0] = pivec[0]   # S1 payoff
        Pay[nMut, 1] = pivec[-1]  # S2 payoff
    
    # Calculate fixation probability using log-sum-exp
    for j in range(1, N):
        # Calculate expected payoffs
        pi1 = 0.0
        pi2 = 0.0
        
        for k in range(n):
            # pi1: S1 player's expected payoff
            pi1 += binom[j-1, k] * Pay[k+1, 0]
            
            # pi2: S2 player's expected payoff
            pi2 += binom[j, k] * Pay[k, 1]
        
        diff = pi1 - pi2
        alpha[j-1] = np.exp(-beta * diff)
    
    # Calculate cumulative product with logarithms for stability
    log_cumprod = np.zeros(N - 1)
    current = 0.0
    for i in range(N - 1):
        current += np.log(alpha[i] if alpha[i] > 0 else 1e-100)
        log_cumprod[i] = current
    
    # Find maximum for numerical stability
    max_val = log_cumprod[0]
    for i in range(1, N - 1):
        if log_cumprod[i] > max_val:
            max_val = log_cumprod[i]
    
    if max_val > 700:  # Avoid overflow
        return 0.0
    
    # Calculate sum of exponentials
    sum_term = 0.0
    for i in range(N - 1):
        sum_term += np.exp(log_cumprod[i] - max_val)
    
    Rho = 1.0 / (1.0 + np.exp(max_val) * sum_term)
    return Rho

# ==================== MAIN NUMBA-OPTIMIZED FUNCTION ====================

@njit(fastmath=True, parallel=True)
def CalcSMEquilibrium(qvec, rvec, c, beta, N=100):
    """
    Selection-mutation equilibrium calculation with full Numba optimization
    """
    n = len(qvec) - 1
    
    # Precompute binomial coefficients
    binom = CalcBinom(N, n)
    
    # Generate all strategies
    Str = get_strategies(n)
    ns = Str.shape[0]
    
    # Calculate homogeneous payoffs
    PayH = np.zeros(ns)
    CoopH = np.zeros(ns)
    
    for i in range(ns):
        # Create homogeneous population
        StrH = np.zeros((n, 2 * n))
        for k in range(n):
            StrH[k] = Str[i]
        
        pivec, cvec = calcPay(StrH, qvec, rvec, c)
        
        PayH[i] = pivec[0]
        CoopH[i] = cvec
    
    # Build transition matrix
    T = np.zeros((ns, ns))
    
    # Calculate transition probabilities
    for i in range(ns):
        row_sum = 0.0
        for j in range(ns):
            if j != i:
                rho = CalcRho(j, i, PayH, N, n, qvec, rvec, c, beta, binom, Str)
                T[i, j] = (1.0 / (ns - 1)) * rho
                row_sum += T[i, j]
        
        T[i, i] = 1.0 - row_sum
    
    # Ensure rows sum to 1
    for i in range(ns):
        row_sum = 0.0
        for j in range(ns):
            row_sum += T[i, j]
        if abs(row_sum - 1.0) > 1e-10:
            for j in range(ns):
                T[i, j] /= row_sum
    
    # Calculate stationary distribution using power iteration
    freq = np.ones(ns) / ns
    for iteration in range(10000):
        freq_new = np.zeros(ns)
        
        # Matrix multiplication
        for i in range(ns):
            for j in range(ns):
                freq_new[i] += freq[j] * T[j, i]
        
        # Normalize
        freq_new_sum = 0.0
        for i in range(ns):
            freq_new_sum += freq_new[i]
        freq_new = freq_new / freq_new_sum
        
        # Check convergence
        max_diff = 0.0
        for i in range(ns):
            diff = abs(freq_new[i] - freq[i])
            if diff > max_diff:
                max_diff = diff
        
        if max_diff < 1e-12:
            break
        
        freq = freq_new
    
    # Normalize final frequencies
    freq_sum = 0.0
    for i in range(ns):
        freq_sum += freq[i]
    freq = freq / freq_sum
    
    # Calculate equilibrium quantities
    coop = 0.0
    pay = 0.0
    for i in range(ns):
        coop += freq[i] * CoopH[i]
        pay += freq[i] * PayH[i]
    
    return coop, pay, freq, Str

In [21]:
def GetEvolData():
    """
    Creates the data for Figure 4 by calling the subroutine
    CalcSMEquilibrium, which computes the cooperation rate of the stochastic
    game in the selection-mutation equilibrium of the stochastic process.
    
    Returns:
    - coop: Average cooperation rate in equilibrium for each scenario
    - pay: Average payoff for each scenario
    - freq: Abundance of each strategy for each scenario
    - Str: List of all strategies
    - rvecG: List of multiplication factors (Gradual scenario)
    - qvec: Vector where q[i] is the next state, given i cooperators
    - Data: Summary of the used parameters
    """
    
    # Start timer
    start_time = time.time()
    
    # Defining the parameters
    beta = 10
    c = 1
    Data = f'c={c}; beta={beta}'
    
    # Defining the four scenarios: Immediate / Gradual / Delayed / None
    rvecI = np.array([1.6, 1.0, 1.0, 1.0, 1.0])      # Immediate
    rvecD = np.array([1.6, 1.6, 1.6, 1.6, 1.0])      # Delayed
    rvecG = np.array([1.6, 1.45, 1.3, 1.15, 1.0])    # Gradual
    rvecN = np.array([1.6, 1.6, 1.6, 1.6, 1.6])      # None
    
    # Transition structure: q[i] is next state given i cooperators
    # Note: In MATLAB, indexing starts at 1, so qvec[1] corresponds to 0 cooperators
    # In Python, qvec[0] will correspond to 0 cooperators
    qvec = np.array([4, 3, 2, 1, 0], dtype=np.int64)
      # States are 1-indexed in original
    
    # Group size n = length(qvec) - 1
    n = len(qvec) - 1
    
    print("Calculating selection-mutation equilibrium for 4 scenarios...")
    print(f"Group size: n = {n}")
    print(f"Beta: {beta}, Cost: c = {c}")
    
    # Calculating the output for each scenario
    print("\n1. Calculating Immediate scenario...")
    coopI, payI, freqI, Str = CalcSMEquilibrium(qvec, rvecI, c, beta)
    
    print("\n2. Calculating Gradual scenario...")
    coopG, payG, freqG, Str = CalcSMEquilibrium(qvec, rvecG, c, beta)
    
    print("\n3. Calculating Delayed scenario...")
    coopD, payD, freqD, Str = CalcSMEquilibrium(qvec, rvecD, c, beta)
    
    print("\n4. Calculating None scenario...")
    coopN, payN, freqN, Str = CalcSMEquilibrium(qvec, rvecN, c, beta)
    
    # Collecting the output into vectors / matrices
    coop = np.array([coopI, coopG, coopD, coopN])
    pay = np.array([payI, payG, payD, payN])
    freq = np.vstack([freqI, freqG, freqD, freqN])
    
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"\nTotal calculation time: {elapsed_time:.2f} seconds")
    
    # Print results
    print("\n" + "="*50)
    print("RESULTS SUMMARY")
    print("="*50)
    scenarios = ["Immediate", "Gradual", "Delayed", "None"]
    for i, scenario in enumerate(scenarios):
        print(f"{scenario:10s}: Cooperation = {coop[i]:.4f}, Payoff = {pay[i]:.4f}")
    
    # Find most abundant strategy for each scenario
    print("\nMost abundant strategies:")
    for i, scenario in enumerate(scenarios):
        max_idx = np.argmax(freq[i])
        max_freq = freq[i, max_idx]
        print(f"{scenario:10s}: Strategy {max_idx} (freq: {max_freq:.4f})")
    
    return coop, pay, freq, Str, rvecG, qvec, Data, elapsed_time

In [ ]:
results = GetEvolData()
    
# Unpack results
coop, pay, freq, Str, rvecG, qvec, Data, elapsed_time = results

print(f"\nData summary: {Data}")
print(f"rvecG (Gradual scenario multiplication factors): {rvecG}")
print(f"qvec (transition structure): {qvec}")

Calculating selection-mutation equilibrium for 4 scenarios...
Group size: n = 4
Beta: 10, Cost: c = 1

1. Calculating Immediate scenario...
